This file downloads satellite imagery using Sentinel API

In [14]:
#!pip install sentinelsat

In [37]:
# imports
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt

import os
from urllib import request
from datetime import date
import datetime
import pandas as pd
import json
import zipfile

In [75]:
# constants
CWD = os.getcwd()
URL = 'https://developers.google.com/public-data/docs/canonical/countries_csv'
COU_DIR = '/data/country/'
IMG_DIR = '/data/image/'
USER = ''
PASS = ''
DATE_END = date.today()

# adjustable search parameters
COUNTRY = 'US' # abbreviation of country. options here: 'https://developers.google.com/public-data/docs/canonical/countries_csv'
YEAR_SEARCH = 1 # how many previous years to query
CLOUD_COVER = [0, 10] # acceptable amount of cloud cover [start, end] (percentage)

# custom lat/long search
LAT, LONG = 47.608013, -122.335167 # custom latitude&longitude (-999 indicates no custom search)
CUS_NAME = 'SEA' # custom location name
CUSTOM = True # False: no custom search, True: custom search


In [60]:
# requests country data from google, returns dataframe of country data
def getCountry(URL):
    webpage = request.urlopen(URL)
    country_data = pd.read_html(webpage)[0]
    return country_data


In [61]:
# get desired country from user input, returns lat/long and country name
def userInput(URL, countries):
    country_ab = COUNTRY
    row = countries[countries.country == country_ab]
    lat, long, cou_name = row.latitude.values[0], row.longitude.values[0], row.country.values[0]
    return(lat, long, cou_name)


In [62]:
# creates geojson from desired lat/long, used for footprint of API query
def createGeoJson(URL, countries):
    if CUSTOM:
        lat, long, cou_name = LAT, LONG, CUS_NAME
    else:
        # get country information
        lat, long, cou_name = userInput(URL, countries)
    
    # construct geojson with lat/long
    data = {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": {
                    "type": "Point", 
                    "coordinates": [long, lat]
                }
            }
        ]
    }
    
    # filepath where .geojson will be saved
    filepath = CWD + COU_DIR + cou_name + '.geojson'
    
    # writes geojson file to directory
    with open(filepath, "w", encoding = 'utf-8') as f:
        json.dump(data, f, ensure_ascii = False, indent = 4) # formatting
    f.close()
    return filepath
    

In [63]:
# determines how many years to query based on YEAR_SEARCH variable, returns date 
def getStartDate():
    DATE_START = DATE_END - pd.DateOffset(years = YEAR_SEARCH)
    return DATE_START


In [64]:
# connect to API service, requests based on search parameters, returns sorted dataframe of products
def sentinel_product(filepath, api):
    footprint = geojson_to_wkt(read_geojson(filepath))

    products = api.query(footprint,
                         date = (getStartDate(), DATE_END),
                         platformname = 'Sentinel-2',
                         cloudcoverpercentage = CLOUD_COVER)


    products_df = api.to_dataframe(products).sort_values(by = ['summary'], ascending = False)

    return products_df


In [65]:
def sentinel_contact():
    api = SentinelAPI(USER, PASS, 'https://apihub.copernicus.eu/apihub')
    return api

In [66]:
# uses product dataframe parameter to download first satellite imagery, returns file title
def sentinel_download(products_df, api):

    title = products_df.iloc[0]['title']
    prod_id = products_df.iloc[0]['uuid']

    api.download(prod_id, CWD + IMG_DIR)
    
    return title


In [67]:
# unpacks downloaded file using title parameter, removes zipped version
def unpack(title):
    
    dl_filepath = CWD + IMG_DIR + title + '.zip'
    
    # unpacking downloaded zip
    with zipfile.ZipFile(dl_filepath, 'r') as zip_ref:
        zip_ref.extractall(CWD + IMG_DIR)
    os.remove(dl_filepath)
    
    # renaming file
    dl_filepath = CWD + IMG_DIR + title + '.SAFE'
    
    if CUSTOM:
        filename = 'sentinel' + CUS_NAME
    else:
        filename = 'sentinel' + COUNTRY
    dl_rename = CWD + IMG_DIR + filename
    os.rename(dl_filepath, dl_rename)
    
    

In [73]:
def main():
    if LAT == -999 and LONG == -999:
        countries = getCountry(URL)
        filepath = createGeoJson(URL, countries)
        api = sentinel_contact()
        products_df = sentinel_product(filepath, api)
        title = sentinel_download(products_df, api)
        unpack(title)
        print("Complete.")
    else:
        filepath = createGeoJson(0, 0)
        api = sentinel_contact()
        products_df = sentinel_product(filepath, api)
        title = sentinel_download(products_df, api)
        unpack(title)
        print("Complete.")

In [76]:
main()


MD5 checksumming:   0%|          | 0.00/902M [00:00<?, ?B/s]

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\Connor\\Desktop\\treeDetection/data/image/S2A_MSIL1C_20211231T191821_N0301_R056_T10TET_20211231T210256.SAFE' -> 'C:\\Users\\Connor\\Desktop\\treeDetection/data/image/sentinelUS'